<a href="https://colab.research.google.com/github/mahyarhabibi/GenderGaps_Hollywood/blob/main/Codes/gender_gaps_agg_pub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook generates the results shown in Table 2 and Table 3 in the paper, estimating gender gaps in box office sales, budgets, critic and user scores ,and the impact of the \#MeToo movement on these outcomes.

In [1]:
seed = 1000

In [2]:
import numpy as np
import pandas as pd 
import scipy as sc
import statsmodels.api as sm
from patsy import dmatrices
import re
import os
import sympy

In [3]:
!pip install -U DoubleML

from doubleml import DoubleMLData
from doubleml import DoubleMLPLR
from doubleml import DoubleMLIRM

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 125 kB 5.0 MB/s 
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=e8acd865754a2a68c9b1051c1b538c04591aa14131388c50421555004aa61221
  Stored in directory: /root/.cache/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built sklearn


In [4]:
from sklearn.base import clone
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression as OLS
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.model_selection import ShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()

from xgboost import XGBClassifier, XGBRegressor

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [6]:
# Directories to read data and store results
#Please add the parent folder to your Google Drive
# Public Link: https://drive.google.com/drive/folders/1TYCDAJOCiLZw4TObcLac5GnL5YtwYnUD?usp=sharing
parent_dir = "/content/gdrive/MyDrive/GenderGaps_Hollywood/" # You may need to change the address
data_dir = os.path.join(parent_dir, 'Data/')
results_dir = os.path.join(parent_dir,'Results/')

In [7]:
df_movies = pd.read_pickle(data_dir + 'movies_info_merged_MIW_pkl.zip')

In [8]:
genres = set(df_movies['genre'].sum())
for gen in genres:
    df_movies[gen] = df_movies['genre'].map(lambda x: 1 if gen in x else 0)
    
df_movies = df_movies.rename(columns={'Film-Noir': 'FilmNoir', 'Sci-Fi': 
                                      'SciFi', 'Reality-TV': 'RealityTV',
                                      'Talk-Show': 'TalkShow'})

genres = genres - {'Film-Noir','Sci-Fi','Reality-TV', 'Talk-Show'}
genres = list(genres.union({'FilmNoir','SciFi','RealityTV', 'TalkShow'}))

In [9]:
# Creating/loading a file for storing the results
import os.path
import json
if os.path.exists(results_dir + 'agg_outcomes_reg_results.json'):
  with open(results_dir + 'agg_outcomes_reg_results.json', 'r') as input_file:
   agg_outcomes_reg_results = json.load(input_file)
else:
  agg_outcomes_reg_results = {}

In [10]:
# Function to store DML results
def store_dml_results(model_res, post_me2=False):
  coef_DFem = model_res.coef[0]
  se_DFem = model_res.se[0]
  pval_DFem = model_res.pval[0]
  if post_me2==True:
    coef_pme2 = model_res.coef[1]
    se_pme2 = model_res.se[1]
    pval_pme2 = model_res.pval[1]
  else: coef_pme2, se_pme2, pval_pme2 = np.nan, np.nan, np.nan

  return {'coef D_Female': [coef_DFem, coef_pme2], 'se D_Female': [se_DFem, se_pme2],
          'p-val D_Female': [pval_DFem, pval_pme2]}

In [11]:
# Function to store OLS results
def store_ols_results(reg_res,loc_DFem, loc_DFem_pme2=None, post_me2=False):
  coef_DFem = reg_res.params[loc_DFem]
  se_DFem = reg_res.bse[loc_DFem]
  pval_DFem = reg_res.pvalues[loc_DFem]
  r2 = reg_res.rsquared
  r2_adj = reg_res.rsquared_adj
  N_obs = reg_res.nobs
  if post_me2==True:
    coef_pme2 = reg_res.params[loc_DFem_pme2]
    se_pme2 = reg_res.bse[loc_DFem_pme2]
    pval_pme2 = reg_res.pvalues[loc_DFem_pme2]
  else: coef_pme2, se_pme2, pval_pme2 = np.nan, np.nan, np.nan

  return {'coef D_Female': [coef_DFem, coef_pme2], 'se D_Female': [se_DFem, se_pme2],
          'p-val D_Female': [pval_DFem, pval_pme2], 'R2': r2, 'Adj R2': r2_adj,
          'N Obs': N_obs}

## Sales

### OLS

In [12]:
# Temporarily Storing Results in dicts
reg_sales_dml_results={}
reg_sales_ols_results = {}

In [13]:
# Removing Movies made by online streaming platforms
online_platforms = ['Netflix', 'Amazon Studios','Hulu','Amazon Prime Video', 
                      'Disney+', 'HBO Max' ]
df_movies['online_plt'] = df_movies['Distributor'].map(lambda x: 1 if x in 
                                                       online_platforms else 0)


In [14]:
df_boxoff = df_movies.loc[~((df_movies['est_sales'].isna()) | 
                            (df_movies['est_budget'].isna()) | 
                            ( df_movies['online_plt']==1))].reset_index(drop=True)

In [15]:
df_boxoff[['Year', 'metascore', 'N_review', 'english', 'made_us', 'D_Female',
           'est_budget', 'est_sales']].describe()

,Year,metascore,N_review,english,made_us,D_Female,est_budget,est_sales
count,5486.000000,5486.000000,5486.000000,5486.000000,5486.000000,5152.000000,5486.000000,5486.000000
mean,2007.039920,54.370944,26.824098,0.941305,0.859825,0.102873,32.011008,85.730269
std,8.016713,17.488687,10.978750,0.235074,0.347200,0.303822,40.256378,167.896973
min,1990.000000,1.000000,7.000000,0.000000,0.000000,0.000000,0.000003,0.000139
25%,2001.000000,42.000000,18.000000,1.000000,1.000000,0.000000,7.000000,4.091704
50%,2008.000000,55.000000,27.000000,1.000000,1.000000,0.000000,18.000000,25.090584
75%,2014.000000,67.000000,35.000000,1.000000,1.000000,0.000000,40.000000,91.744965
max,2021.000000,100.000000,67.000000,1.000000,1.000000,1.000000,317.000000,2847.246203


In [16]:
# XS_base: X_base of the Sales specification
y_sales, XS_base = dmatrices('np.log(est_sales) ~  D_Female + C(Year)',
                    data=df_boxoff, return_type='dataframe')

# XGB throws error if '[]' are in columns names:
year_cols = dict(zip([f'C(Year)[T.{y}]' for y in range(1991,2022)],
                     [f'Year({y})' for y in range(1991,2022)] ))
XS_base = XS_base.rename(columns = year_cols)

In [17]:
# Mean Dependant Variable
y_sales.describe()

,np.log(est_sales)
count,5152.000000
mean,2.719025
std,2.515618
min,-7.377759
25%,1.454029
50%,3.252073
75%,4.516292
max,7.954108


In [18]:
# OLS Sales Baseline
y_ols = y_sales.copy()
X_ols = XS_base.copy()
ols = sm.OLS(y_ols, X_ols) 
reg_res = ols.fit().get_robustcov_results()

loc_DFem = X_ols.columns.get_loc('D_Female')
reg_sales_ols_results.update({'Base': store_ols_results(reg_res,loc_DFem)})

In [19]:
#Countries
df_boxoff['country_list'] = df_boxoff['country'].map(lambda x: re.sub(' ','',x)).\
                        map(lambda x: x.split(','))

X_country = pd.DataFrame(mlb.fit_transform(df_boxoff['country_list']),
             columns=mlb.classes_, index=df_boxoff.index)

X_country = X_country[X_country.columns[X_country.sum()>=10]]

#Language
df_boxoff['lang_list'] = df_boxoff['language'].map(lambda x: re.sub(' ','',x)).\
                        map(lambda x: x.split(','))
X_lang = pd.DataFrame(mlb.fit_transform(df_boxoff['lang_list']),
             columns=mlb.classes_, index=df_boxoff.index)
X_lang = X_lang[X_lang.columns[X_lang.sum()>=10]]

In [20]:
# OLS Country and Language FE: Spec2
y_ols = y_sales.copy()
X_ols = XS_base.join(X_country, rsuffix='cn').join(X_lang, rsuffix='lg')

ols = sm.OLS(y_ols, X_ols) 
reg_res = ols.fit().get_robustcov_results()

loc_DFem = X_ols.columns.get_loc('D_Female')
reg_sales_ols_results.update({'spec2': store_ols_results(reg_res,loc_DFem)})

In [21]:
# Plot Truncated SVD Vectors
plot_vectors = [f'plot_vec{i}' for i in range(100)]
X_vecs = df_boxoff[plot_vectors]

In [22]:
# spec3 = spec2 + Plot Vectors FE
y_ols = y_sales.copy()
X_ols = XS_base.join(X_country, rsuffix='cn').join(X_lang, rsuffix='lg').join(X_vecs)

ols = sm.OLS(y_ols, X_ols) 
reg_res = ols.fit().get_robustcov_results()

loc_DFem = X_ols.columns.get_loc('D_Female')
reg_sales_ols_results.update({'spec3': store_ols_results(reg_res,loc_DFem)})

In [23]:
# Companies with at least 10 movies
X_company = pd.get_dummies(df_boxoff['company'])
X_company = X_company[X_company.columns[X_company.sum()>=10]]

# Distributors with at least 10 movies
X_dist = pd.get_dummies(df_boxoff['Distributor'])
X_dist = X_dist[X_dist.columns[X_dist.sum()>=10]]

In [24]:
# spec4:  spec3 + company and distributor FE
y_ols = y_sales.copy()
X_ols = XS_base.join(X_country, rsuffix='cn').join(X_lang, rsuffix='lg').\
        join(X_vecs).join(X_company, rsuffix='cp').\
        join(X_dist, rsuffix='ds')

ols = sm.OLS(y_ols, X_ols) 
reg_res = ols.fit().get_robustcov_results()

loc_DFem = X_ols.columns.get_loc('D_Female')
reg_sales_ols_results.update({'spec4': store_ols_results(reg_res,loc_DFem)})

In [25]:
#Cast
X_cast = pd.DataFrame(mlb.fit_transform(df_boxoff['cast']),columns=mlb.classes_, index=df_boxoff.index)
# Keeping actors whose names showed up at least 10 times
X_cast = X_cast[X_cast.columns[X_cast.sum()>=10]]

# Age Ratings Categories
X_mpaa = df_boxoff[['rated_R','rated_PG', 'rated_PG13', 'rated_TVMA', 'rated_TV14']]

# Genres
X_genre = df_boxoff[genres]
# X_genre is not full rank in sales and budget data
_, inds = sympy.Matrix(X_genre.values).rref() # returns independent indices
keep_ind = list(inds)
X_genre = X_genre.iloc[:,keep_ind] # full-rank

In [26]:
# spec5:  spec4 + cast + age ratigns, mpaa, and genres
y_ols = y_sales.copy()
X_ols = XS_base.join(X_country, rsuffix='cn').join(X_lang, rsuffix='lg').\
        join(X_vecs).join(X_cast, rsuffix='cs').join(X_company, rsuffix='cp').\
        join(X_dist, rsuffix='ds').join(X_mpaa).join(X_genre, rsuffix='gn')

ols = sm.OLS(y_ols, X_ols) 
reg_res = ols.fit().get_robustcov_results()

loc_DFem = X_ols.columns.get_loc('D_Female')
reg_sales_ols_results.update({'spec5': store_ols_results(reg_res,loc_DFem)})

In [123]:
reg_sales_ols_results['spec5']

{'coef D_Female': [-0.15706103324758702, nan],
 'se D_Female': [0.08466112788589993, nan],
 'p-val D_Female': [0.06363980101081147, nan],
 'R2': 0.7127836549652102,
 'Adj R2': 0.6565804565287368,
 'N Obs': 5152.0}

### DML

In [27]:
# Using the complete set of features to estimate DML models
XS_full = XS_base.join(X_country, rsuffix='cn').join(X_lang, rsuffix='lg').\
        join(X_vecs).join(X_cast, rsuffix='cs').join(X_company, rsuffix='cp').\
        join(X_dist, rsuffix='ds').join(X_mpaa).join(X_genre, rsuffix='gn')

data_sales = XS_full.join(y_sales)
y_sales_dml = data_sales['np.log(est_sales)']
data_sales = data_sales.rename(columns={'np.log(est_sales)': 'log_sales'})
x_cols = list(set(data_sales.columns.values) - {'D_Female','log_sales' })

dml_data_sales = DoubleMLData(data_sales, y_col = 'log_sales', d_cols = 'D_Female',
                        x_cols = x_cols)

print(f'Number of Features: {len(x_cols)}')

Number of Features: 843


In [28]:
# Since tuning models in DML takes time, I specified the tuned versions from the previous runs.
# The codes for tuning the classifers can be found below.

lgt_classifier = LogisticRegression(C=1000000.0, penalty='l1', random_state=1000,
                   solver='liblinear')

lgt_params = {'C': [1000000.0], 'penalty': ['l1']}

ENet_tuned = ElasticNet(alpha=0.001, l1_ratio=0.75)

RF_tuned = RandomForestRegressor(n_jobs = -1, n_estimators = 300, max_features = 'sqrt'
                          , max_depth=80, min_samples_split= 8, random_state=seed)

XGB_tuned = XGBRegressor(objective = "reg:squarederror", eta = 0.2,
                        n_estimators =50, n_jobs=-1, max_depth= 10, random_state=seed)

 **NOTE**: DML uses the accuracy metric to tune the binary classifiern and not F1 score. 
 Considering the accuracy metric, the model is best tuned when it predicts D_Female = 0
 for all movies to obtain a accuracy of 90%, but with a F1 equal to 0. Therefore, 
 I tune the binary classifier outside of the DML Class. 

In [29]:
# # Tuning Classifier (learner_m):
# # Selection Stage

# d_fem = XS_full['D_Female']
# X_cls = XS_full.drop(columns=['D_Female'])

# param_grid = {'penalty': ['l2', 'l1'], 
#               'C': [ 1e5,1e6, 1e7, 1e8, 1e9]}

# # Logit
# lgt = LogisticRegression(solver='liblinear', random_state=seed)
# clf = GridSearchCV(lgt, param_grid, scoring='f1',cv = ShuffleSplit(n_splits=5,random_state=seed))
# clf.fit(X_cls, d_fem)
# lgt_classifier = clf.best_estimator_
# lgt_params = {'C': [clf.best_params_['C']], 'penalty': [clf.best_params_['penalty']]}

# # F1 Test Scores are about 0.20
# lgt_test_scores = cross_val_score(lgt_classifier,X_cls, d_fem, n_jobs=-1, scoring = 'f1',cv = ShuffleSplit(n_splits=5))
# print(f'lgt_scores: {lgt_test_scores}')

In [30]:
# # Random Forests
# rfc_scores = cross_val_score(RF_tuned,XS_full, y_sales, n_jobs=-1, scoring = 'r2',
#                              cv = ShuffleSplit(n_splits=3))
# print(f'rfc_scores: {rfc_scores}')

# # XGBoost: TFIDF Matrix
# xgb_scores =  cross_val_score(XGB_tuned,XS_full, y_sales, n_jobs=-1, scoring = 'r2',
#                               cv = ShuffleSplit(n_splits=3))
# print(f'xgb_scores: {xgb_scores}')

# # Elastic Net: 
# elnet_scores = cross_val_score(ENet_tuned,XS_full, y_sales, n_jobs=-1, scoring = 'r2',cv = ShuffleSplit(n_splits=5))
# print(f'elnet_scores: {elnet_scores}')

# # CV R2 Summary: RF roughly 0.45, XGB roughly 0.50, Elastic Net: roughly 0.63

In [31]:
# DML 1
np.random.seed(seed)
# learner_g = ElasticNet()
learner_g = ENet_tuned
learner_m = lgt_classifier
ml_g = clone(learner_g)
ml_m = clone(learner_m)
param_grids = {'ml_g': {'alpha': [0.0001, 0.001, 0.1], 'l1_ratio': [ 0.01,0.25,0.5,0.75,0.99]},
              'ml_m': lgt_params}
dml_sales = DoubleMLPLR(dml_data_sales, ml_g, ml_m)
# dml_sales.tune(param_grids, n_folds_tune=3, n_jobs_cv=-1)
dml_sales.fit(n_jobs_cv=-1, store_predictions=True);

# Store Coef and SE
reg_sales_dml_results.update({'ElasticNet': store_dml_results(dml_sales)})

In [32]:
# DML Random Forest
np.random.seed(seed)
# learner_g = RandomForestRegressor(n_jobs = -1, n_estimators = 300,
#                                   max_features= 'sqrt')
learner_g = RF_tuned
learner_m = lgt_classifier
ml_g = clone(learner_g)
ml_m = clone(learner_m)

param_grids = {'ml_g': {'max_depth': [ 20, 40, 80], 'min_samples_split': [4,8,16]},
              'ml_m': lgt_params}

dml_sales = DoubleMLPLR(dml_data_sales, ml_g, ml_m)
# dml_sales.tune(param_grids, n_folds_tune=3, n_jobs_cv=-1)
dml_sales.fit(n_jobs_cv=-1, store_predictions=True);

# Store Coef and SE
reg_sales_dml_results.update({'RandomForest': store_dml_results(dml_sales)})

In [33]:
# DML XGBoost
np.random.seed(seed)
# learner_g =  XGBRegressor(objective = "reg:squarederror",
#                         n_estimators =50, n_jobs=-1, random_state=seed)
learner_g = XGB_tuned
learner_m = lgt_classifier
ml_g = clone(learner_g)
ml_m = clone(learner_m)
param_grids = {'ml_g': {'max_depth': [10, 20, 30], 'eta': [0.2, 0.4, 0.6]},
              'ml_m': lgt_params}

dml_sales = DoubleMLPLR(dml_data_sales, ml_g, ml_m)
# dml_sales.tune(param_grids, n_folds_tune=3, n_jobs_cv=-1)
dml_sales.fit(n_jobs_cv=-1, store_predictions=True);

# Store Coef and SE
reg_sales_dml_results.update({'XGBoost': store_dml_results(dml_sales)})

### Winsorized Regs

In [34]:
q01_sales = y_sales['np.log(est_sales)'].quantile(.01)
q99_sales = y_sales['np.log(est_sales)'].quantile(.99)

keep_indx = y_sales.loc[((y_sales['np.log(est_sales)']>q01_sales) & 
          (y_sales['np.log(est_sales)']<q99_sales))].index.values

y_sales_win = y_sales.iloc[y_sales.index.isin(keep_indx)]
XS_base_win = XS_base.iloc[y_sales.index.isin(keep_indx)]

In [35]:
# Winsorized: full controls
y_ols = y_sales_win.copy()
X_ols = XS_base_win.join(X_country, rsuffix='cn').join(X_lang, rsuffix='lg').\
        join(X_vecs).join(X_cast, rsuffix='cs').join(X_company, rsuffix='cp').\
        join(X_dist, rsuffix='ds').join(X_mpaa).join(X_genre, rsuffix='gn')

ols = sm.OLS(y_ols, X_ols) 
reg_res = ols.fit().get_robustcov_results()

loc_DFem = X_ols.columns.get_loc('D_Female')
reg_sales_ols_results.update({'winsorized_reg': store_ols_results(reg_res,loc_DFem)})

### Quantile Regression

In [36]:
ys_qreg = y_sales.copy()
XS_qreg = XS_base.join(X_country, rsuffix='cn').join(X_lang, rsuffix='lg').\
        join(X_vecs).join(X_cast, rsuffix='cs').join(X_company, rsuffix='cp').\
        join(X_dist, rsuffix='ds').join(X_mpaa).join(X_genre, rsuffix='gn')

In [37]:
# Defining quantile regression models
qreg_model = sm.regression.quantile_regression.QuantReg(ys_qreg,XS_qreg)
def qreg_fit(q):
  res= qreg_model.fit(q)
  return [q, res.params['D_Female'], res.bse['D_Female'], res.pvalues['D_Female']
          ] +   res.conf_int().loc['D_Female'].tolist() 


In [38]:
# # Estiamting quantile regression for q=0.1, 0.2, .... 0.9
# # Takes a LONG time
# quantiles = np.arange(0.1, 0.91,0.10)
# qreg_results = [qreg_fit(q) for q in quantiles]
# df_qreg_res = pd.DataFrame(qreg_results, columns=["q", "coef", "se", "p-val", "lb", "ub"])
# df_qreg_res.to_csv(results_dir + 'qreg_results_sales.csv')

### MeToo Impact

In [39]:
df_boxoff_me2= df_boxoff.copy()
#post me too >=2018
df_boxoff_me2['DFem_PMe2'] = np.where(((df_boxoff_me2['D_Female']==1) & (df_boxoff_me2['Year']>2017)), 1,0)

In [40]:
# OLS
# XS_base: X_base of the Sales specification
y_sales, XS_me2 = dmatrices('np.log(est_sales) ~ D_Female + C(Year) + DFem_PMe2',
                    data=df_boxoff_me2, return_type='dataframe')

# OLS: Fully Saturated Model
y_ols = y_sales.copy()
X_ols = XS_me2.join(X_country, rsuffix='cn').join(X_lang, rsuffix='lg').\
        join(X_vecs).join(X_cast, rsuffix='cs').join(X_company, rsuffix='cp').\
        join(X_dist, rsuffix='ds').join(X_mpaa).join(X_genre, rsuffix='gn')

ols = sm.OLS(y_ols, X_ols) 
reg_res = ols.fit().get_robustcov_results()

loc_DFem = X_ols.columns.get_loc('D_Female')
loc_DFem_pme2 = X_ols.columns.get_loc('DFem_PMe2')
reg_sales_ols_results.update({'metoo': 
                      store_ols_results(reg_res,loc_DFem,loc_DFem_pme2,
                                        post_me2=True)})

In [41]:
# DML
data_sales_me2 = X_ols.join(y_sales)
data_sales_me2 = data_sales_me2.rename(columns={'np.log(est_sales)': 'log_sales'})
x_cols_me2 = list(set(X_ols.columns.values) - {'D_Female', 'DFem_PMe2'})

dml_data_sales_me2 = DoubleMLData(data_sales_me2, y_col = 'log_sales', d_cols =
                                 ['D_Female','DFem_PMe2'], x_cols = x_cols_me2)

# Double ML:
# DML ElasticNet
learner_g = ENet_tuned
learner_m = lgt_classifier
ml_g = clone(learner_g)
ml_m = clone(learner_m)
dml_sales = DoubleMLPLR(dml_data_sales_me2, ml_g, ml_m)
dml_sales.fit(n_jobs_cv=-1, store_predictions=True);
reg_sales_dml_results.update({'metoo(ENet)': store_dml_results(dml_sales, 
                                                                   post_me2=True)})

### Save Results

In [126]:
#Save Results
reg_sales_results = {'OLS': reg_sales_ols_results, 'DML': reg_sales_dml_results}
agg_outcomes_reg_results.update({'Sales': reg_sales_results})

with open(results_dir + 'agg_outcomes_reg_results.json', 'w') as outfile:
    json.dump(agg_outcomes_reg_results, outfile)

In [43]:
# Create DataFrame of the Results

specs = list(reg_sales_ols_results.keys()) + list(reg_sales_dml_results.keys()) 

params_ols =  ['coef D_Female', 'se D_Female', 'p-val D_Female']
params_dml =  ['coef D_Female', 'se D_Female', 'p-val D_Female']
spec_ols = list(reg_sales_ols_results.keys() -{'metoo'})
spec_dml = list(reg_sales_dml_results.keys() - {'metoo(ENet)'} )

data_ols = [[reg_sales_ols_results[spec][p][0] for p in params_ols] for spec in spec_ols]
data_r2 = [[reg_sales_ols_results[spec][p] for p in ['R2', 'N Obs']] for spec in spec_ols]
data_ols = [data_ols[r] + data_r2[r] for r in range(len(data_ols))]

data_dml = [[reg_sales_dml_results[spec][p][0] for p in params_dml] for spec in spec_dml]

data_DFem = np.asarray(data_ols + data_dml )

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  app.launch_new_instance()


In [44]:
# Cast the results in tables
table_ols_sales = pd.DataFrame(np.asarray(data_ols).T, columns= spec_ols,
                               index = ['coef', 'SE', 'P-Val', 'R2', 'N'])

table_dml_sales = pd.DataFrame(np.asarray(data_dml).T, columns= spec_dml,
                               index = ['coef', 'SE', 'P-Val'])

## Budget

In [45]:
# Temporarily Storing Results
reg_budget_dml_results={}
reg_budget_ols_results={}

### OLS

In [46]:
# XS_base: X_base of the Sales specification
y_budget, XB_base = dmatrices('np.log(est_budget) ~  D_Female + C(Year)',
                    data=df_boxoff, return_type='dataframe')

# XGB throws error if '[]' are in columns names:
year_cols = dict(zip([f'C(Year)[T.{y}]' for y in range(1991,2022)],
                     [f'Year({y})' for y in range(1991,2022)] ))
XB_base = XB_base.rename(columns = year_cols)

In [47]:
# OLS Sales Baseline
y_ols = y_budget.copy()
X_ols = XB_base.copy()
ols = sm.OLS(y_ols, X_ols) 
reg_res = ols.fit().get_robustcov_results()

loc_DFem = X_ols.columns.get_loc('D_Female')
reg_budget_ols_results.update({'Base': store_ols_results(reg_res,loc_DFem)})

In [48]:
#Countries
df_boxoff['country_list'] = df_boxoff['country'].map(lambda x: re.sub(' ','',x)).\
                        map(lambda x: x.split(','))

X_country = pd.DataFrame(mlb.fit_transform(df_boxoff['country_list']),
             columns=mlb.classes_, index=df_boxoff.index)

X_country = X_country[X_country.columns[X_country.sum()>=10]]

#Language
df_boxoff['lang_list'] = df_boxoff['language'].map(lambda x: re.sub(' ','',x)).\
                        map(lambda x: x.split(','))
X_lang = pd.DataFrame(mlb.fit_transform(df_boxoff['lang_list']),
             columns=mlb.classes_, index=df_boxoff.index)
X_lang = X_lang[X_lang.columns[X_lang.sum()>=10]]

In [49]:
# OLS Country and Language FE: Spec2
y_ols = y_budget.copy()
X_ols = XS_base.join(X_country, rsuffix='cn').join(X_lang, rsuffix='lg')

ols = sm.OLS(y_ols, X_ols) 
reg_res = ols.fit().get_robustcov_results()

loc_DFem = X_ols.columns.get_loc('D_Female')
reg_budget_ols_results.update({'spec2': store_ols_results(reg_res,loc_DFem)})

In [50]:
# Plot Truncated SVD Vectors
plot_vectors = [f'plot_vec{i}' for i in range(100)]
X_vecs = df_boxoff[plot_vectors]

In [51]:
# spec3 = spec2 + Plot Vectors FE
y_ols = y_budget.copy()
X_ols = XS_base.join(X_country, rsuffix='cn').join(X_lang, rsuffix='lg').join(X_vecs)

ols = sm.OLS(y_ols, X_ols) 
reg_res = ols.fit().get_robustcov_results()

loc_DFem = X_ols.columns.get_loc('D_Female')
reg_budget_ols_results.update({'spec3': store_ols_results(reg_res,loc_DFem)})

In [52]:
# Companies with at least 10 movies
X_company = pd.get_dummies(df_boxoff['company'])
X_company = X_company[X_company.columns[X_company.sum()>=10]]

# Distributors with at least 10 movies
X_dist = pd.get_dummies(df_boxoff['Distributor'])
X_dist = X_dist[X_dist.columns[X_dist.sum()>=10]]



In [53]:
# spec4:  spec3 + company and distributorFE
y_ols = y_budget.copy()
X_ols = XS_base.join(X_country, rsuffix='cn').join(X_lang, rsuffix='lg').\
        join(X_vecs).join(X_company, rsuffix='cp').\
        join(X_dist, rsuffix='ds')

ols = sm.OLS(y_ols, X_ols) 
reg_res = ols.fit().get_robustcov_results()

loc_DFem = X_ols.columns.get_loc('D_Female')
reg_budget_ols_results.update({'spec4': store_ols_results(reg_res,loc_DFem)})

In [54]:
#Cast
X_cast = pd.DataFrame(mlb.fit_transform(df_boxoff['cast']),columns=mlb.classes_, index=df_boxoff.index)
# Keeping actors whose names showed up at least 10 times
X_cast = X_cast[X_cast.columns[X_cast.sum()>=10]]

# Age Ratings Categories
X_mpaa = df_boxoff[['rated_R','rated_PG', 'rated_PG13', 'rated_TVMA', 'rated_TV14']]

# Genres
X_genre = df_boxoff[genres]
# X_genre is not full rank in sales and budget data
_, inds = sympy.Matrix(X_genre.values).rref() # returns independent indices
keep_ind = list(inds)
X_genre = X_genre.iloc[:,keep_ind] # full-rank

In [55]:
# spec5:  spec4 + cast +  age ratigns, mpaa, and genres
y_ols = y_budget.copy()
X_ols = XS_base.join(X_country, rsuffix='cn').join(X_lang, rsuffix='lg').\
        join(X_vecs).join(X_company, rsuffix='cp').\
        join(X_dist, rsuffix='ds').join(X_mpaa).join(X_genre, rsuffix='gn').\
        join(X_cast, rsuffix='cs')

ols = sm.OLS(y_ols, X_ols) 
reg_res = ols.fit().get_robustcov_results()

loc_DFem = X_ols.columns.get_loc('D_Female')
reg_budget_ols_results.update({'spec5': store_ols_results(reg_res,loc_DFem)})

### DML

In [56]:

XB_full = XB_base.join(X_country, rsuffix='cn').join(X_lang, rsuffix='lg').\
        join(X_vecs).join(X_company, rsuffix='cp').\
        join(X_dist, rsuffix='ds').join(X_mpaa).join(X_genre, rsuffix='gn')

data_budget = XB_full.join(y_budget)
y_budget_dml = data_budget['np.log(est_budget)']
data_budget = data_budget.rename(columns={'np.log(est_budget)': 'log_budget'})
x_cols = list(set(data_budget.columns.values) - {'D_Female','log_budget' })

dml_data_budget = DoubleMLData(data_budget, y_col = 'log_budget', d_cols = 'D_Female',
                        x_cols = x_cols)

print(f'Number of Features: {len(x_cols)}')

Number of Features: 400


In [57]:
# Since tuning models in DML takes time, I defined the tuned versions from the previous runs
ENet_tuned = ElasticNet(alpha=0.001, l1_ratio=0.01)

RF_tuned = RandomForestRegressor(n_jobs = -1, n_estimators = 300, max_features = 'sqrt'
                          , max_depth=40, min_samples_split= 4, random_state=seed)

XGB_tuned = XGBRegressor(objective = "reg:squarederror", eta = 0.2,
                        n_estimators =50, n_jobs=-1, max_depth= 10, random_state=seed)

In [58]:
# # Random Forests
# rfc_scores = cross_val_score(RF_tuned,XB_full, y_budget, n_jobs=-1, scoring = 'r2',
#                              cv = ShuffleSplit(n_splits=3))
# print(f'rfc_scores: {rfc_scores}')

# # XGBoost: TFIDF Matrix
# xgb_scores =  cross_val_score(XGB_tuned,XB_full, y_budget, n_jobs=-1, scoring = 'r2',
#                               cv = ShuffleSplit(n_splits=3))
# print(f'xgb_scores: {xgb_scores}')

# # Elastic Net: 
# elnet_scores = cross_val_score(ENet_tuned,XB_full, y_budget, n_jobs=-1, scoring = 'r2',cv = ShuffleSplit(n_splits=5))
# print(f'elnet_scores: {elnet_scores}')

# # CV R2 Summary: RF roughly 0.40, XGB roughly 0.43, Elastic Net: roughly 0.55

In [59]:
# DML 1
np.random.seed(seed)
# learner_g = ElasticNet()
learner_g = ENet_tuned
learner_m = lgt_classifier
ml_g = clone(learner_g)
ml_m = clone(learner_m)
param_grids = {'ml_g': {'alpha': [0.0001, 0.001, 0.1], 'l1_ratio': [ 0.01,0.25,0.5,0.75,0.99]},
              'ml_m': lgt_params}
dml_budget = DoubleMLPLR(dml_data_budget, ml_g, ml_m)
# dml_budget.tune(param_grids, n_folds_tune=3, n_jobs_cv=-1)
dml_budget.fit(n_jobs_cv=-1, store_predictions=True);

# Store Coef and SE
reg_budget_dml_results.update({'ElasticNet': store_dml_results(dml_budget)})

In [60]:
# DML Random Forest
np.random.seed(seed)
learner_g = RandomForestRegressor(n_jobs = -1, n_estimators = 300,
                                  max_features= 'sqrt')
learner_g = RF_tuned
learner_m = lgt_classifier
ml_g = clone(learner_g)
ml_m = clone(learner_m)

param_grids = {'ml_g': {'max_depth': [ 20, 40, 80], 'min_samples_split': [4,8,16]},
              'ml_m': lgt_params}

dml_budget = DoubleMLPLR(dml_data_budget, ml_g, ml_m)
dml_budget.tune(param_grids, n_folds_tune=3, n_jobs_cv=-1)
dml_budget.fit(n_jobs_cv=-1, store_predictions=True);

# Store Coef and SE
reg_budget_dml_results.update({'RandomForest': store_dml_results(dml_budget)})

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: Learner ml_g was renamed to ml_l. Please adapt the key of param_grids accordingly. The provided param_grids for ml_g are set for ml_l. The redirection will be removed in a future version.
  


In [61]:
# DML XGBoost
np.random.seed(seed)
# learner_g =  XGBRegressor(objective = "reg:squarederror",
#                         n_estimators =50, n_jobs=-1, random_state=seed)
learner_g = XGB_tuned
learner_m = lgt_classifier
ml_g = clone(learner_g)
ml_m = clone(learner_m)
param_grids = {'ml_g': {'max_depth': [10, 20, 30], 'eta': [0.2, 0.4, 0.6]},
              'ml_m': lgt_params}

dml_budget = DoubleMLPLR(dml_data_budget, ml_g, ml_m)
# dml_budget.tune(param_grids, n_folds_tune=3, n_jobs_cv=-1)
dml_budget.fit(n_jobs_cv=-1, store_predictions=True);

# Store Coef and SE
reg_budget_dml_results.update({'XGBoost': store_dml_results(dml_budget)})

### Winsorizred Regs

In [62]:
q01_budget = y_budget['np.log(est_budget)'].quantile(.01)
q99_budget = y_budget['np.log(est_budget)'].quantile(.99)

keep_indx = y_budget.loc[((y_budget['np.log(est_budget)']>q01_budget) & 
          (y_budget['np.log(est_budget)']<q99_budget))].index.values

y_budget_win = y_budget.iloc[y_budget.index.isin(keep_indx)]
XB_base_win = XB_base.iloc[y_budget.index.isin(keep_indx)]

In [63]:
# Winsorized Reg
y_ols = y_budget_win.copy()
X_ols = XB_base_win.join(X_country, rsuffix='cn').join(X_lang, rsuffix='lg').\
        join(X_vecs).join(X_company, rsuffix='cp').\
        join(X_dist, rsuffix='ds').join(X_mpaa).join(X_genre, rsuffix='gn')

ols = sm.OLS(y_ols, X_ols) 
reg_res = ols.fit().get_robustcov_results()

loc_DFem = X_ols.columns.get_loc('D_Female')
reg_budget_ols_results.update({'winsorized_reg': store_ols_results(reg_res,loc_DFem)})

### Quantile Regressions

In [64]:
yb_qreg = y_budget.copy()
XB_qreg = XB_base.join(X_country, rsuffix='cn').join(X_lang, rsuffix='lg').\
        join(X_vecs).join(X_company, rsuffix='cp').\
        join(X_dist, rsuffix='ds').join(X_mpaa).join(X_genre, rsuffix='gn')

In [65]:
# # Caution: Takes LONG to execute
# qreg_model = sm.regression.quantile_regression.QuantReg(yb_qreg,XB_qreg)
# quantiles = np.arange(0.1, 0.91,0.10)
# qreg_results_budg = [qreg_fit(q) for q in quantiles]
# df_qreg_res_budg = pd.DataFrame(qreg_results_budg, columns=["q", "coef", "se", "p-val", "lb", "ub"])
# df_qreg_res_budg.to_csv(results_dir + 'qreg_results_budget.csv')

### MeToo Impact

In [66]:
# OLS
# XS_base: X_base of the Sales specification
y_budget, XB_me2 = dmatrices('np.log(est_budget) ~ D_Female + C(Year) + DFem_PMe2',
                    data=df_boxoff_me2, return_type='dataframe')

# OLS: Fully Saturated Model
y_ols = y_budget.copy()
X_ols = XB_me2.join(X_country, rsuffix='cn').join(X_lang, rsuffix='lg').\
        join(X_vecs).join(X_company, rsuffix='cp').\
        join(X_dist, rsuffix='ds')

ols = sm.OLS(y_ols, X_ols) 
reg_res = ols.fit().get_robustcov_results()

loc_DFem = X_ols.columns.get_loc('D_Female')
loc_DFem_pme2 = X_ols.columns.get_loc('DFem_PMe2')
reg_budget_ols_results.update({'metoo': 
                      store_ols_results(reg_res,loc_DFem,loc_DFem_pme2,
                                        post_me2=True)})

In [67]:
# DML
data_budget_me2 = X_ols.join(y_budget)
data_budget_me2 = data_budget_me2.rename(columns={'np.log(est_budget)': 'log_budget'})
x_cols_me2 = list(set(X_ols.columns.values) - {'D_Female', 'DFem_PMe2'})

dml_data_budget_me2 = DoubleMLData(data_budget_me2, y_col = 'log_budget', d_cols =
                                 ['D_Female','DFem_PMe2'], x_cols = x_cols_me2)

# Double ML:
# DML ElasticNet
learner_g = ENet_tuned
learner_m = lgt_classifier
ml_g = clone(learner_g)
ml_m = clone(learner_m)
dml_budget = DoubleMLPLR(dml_data_budget_me2, ml_g, ml_m)
dml_budget.fit(n_jobs_cv=-1, store_predictions=True);
reg_budget_dml_results.update({'metoo(ENet)': store_dml_results(dml_budget, 
                                                                   post_me2=True)})

### Save Results

In [68]:
#Save Results
reg_budget_results = {'OLS': reg_budget_ols_results, 'DML': reg_budget_dml_results}
agg_outcomes_reg_results.update({'Budget': reg_budget_results})

with open(results_dir + 'agg_outcomes_reg_results.json', 'w') as outfile:
    json.dump(agg_outcomes_reg_results, outfile)

In [69]:
# Create DataFrame of the Results

specs = list(reg_budget_ols_results.keys()) + list(reg_budget_dml_results.keys()) 

params_ols =  ['coef D_Female', 'se D_Female', 'p-val D_Female']
params_dml =  ['coef D_Female', 'se D_Female', 'p-val D_Female']
spec_ols = list(reg_budget_ols_results.keys() -{'metoo'})
spec_dml = list(reg_budget_dml_results.keys() - {'metoo(ENet)'} )

data_ols = [[reg_budget_ols_results[spec][p][0] for p in params_ols] for spec in spec_ols]
data_r2 = [[reg_budget_ols_results[spec][p] for p in ['R2', 'N Obs']] for spec in spec_ols]
data_ols = [data_ols[r] + data_r2[r] for r in range(len(data_ols))]

data_dml = [[reg_budget_dml_results[spec][p][0] for p in params_dml] for spec in spec_dml]


data_DFem = np.asarray(data_ols + data_dml )

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [70]:
# Cast the results in tables
table_ols_budget = pd.DataFrame(np.asarray(data_ols).T, columns= spec_ols,
                               index = ['coef', 'SE', 'P-Val', 'R2', 'N'])

table_dml_budget = pd.DataFrame(np.asarray(data_dml).T, columns= spec_dml,
                               index = ['coef', 'SE', 'P-Val'])

## Metascore

In [71]:
# Temporarily Storing Results in dicts
reg_meta_dml_results={}
reg_meta_ols_results = {}

In [72]:
df_scores = df_movies.loc[~((df_movies['metascore'].isna()) |
                            (df_movies['userscore'].isna()))].reset_index(drop=True)

### OLS

In [73]:
# XS_base: X_base of the meta specification
y_meta, XM_base = dmatrices('metascore ~  D_Female + C(Year)',
                    data=df_scores, return_type='dataframe')

# XGB throws error if '[]' are in columns names:
year_cols = dict(zip([f'C(Year)[T.{y}]' for y in range(1991,2022)],
                     [f'Year({y})' for y in range(1991,2022)] ))
XM_base = XM_base.rename(columns = year_cols)

In [74]:
# OLS meta Baseline
y_ols = y_meta.copy()
X_ols = XM_base.copy()
ols = sm.OLS(y_ols, X_ols) 
reg_res = ols.fit().get_robustcov_results()

loc_DFem = X_ols.columns.get_loc('D_Female')
reg_meta_ols_results.update({'Base': store_ols_results(reg_res,loc_DFem)})

In [75]:
#Countries
df_scores['country_list'] = df_scores['country'].map(lambda x: re.sub(' ','',x)).\
                        map(lambda x: x.split(','))

X_country = pd.DataFrame(mlb.fit_transform(df_scores['country_list']),
             columns=mlb.classes_, index=df_scores.index)

X_country = X_country[X_country.columns[X_country.sum()>=10]]

#Language
df_scores['lang_list'] = df_scores['language'].map(lambda x: re.sub(' ','',x)).\
                        map(lambda x: x.split(','))
X_lang = pd.DataFrame(mlb.fit_transform(df_scores['lang_list']),
             columns=mlb.classes_, index=df_scores.index)
X_lang = X_lang[X_lang.columns[X_lang.sum()>=10]]

In [76]:
# OLS Country and Language FE: Spec2
y_ols = y_meta.copy()
X_ols = XM_base.join(X_country, rsuffix='cn').join(X_lang, rsuffix='lg')

ols = sm.OLS(y_ols, X_ols) 
reg_res = ols.fit().get_robustcov_results()

loc_DFem = X_ols.columns.get_loc('D_Female')
reg_meta_ols_results.update({'spec2': store_ols_results(reg_res,loc_DFem)})

In [77]:
# Plot Truncated SVD Vectors
plot_vectors = [f'plot_vec{i}' for i in range(100)]
X_vecs = df_scores[plot_vectors]

In [78]:
# spec3 = spec2 + Plot Vectors FE
y_ols = y_meta.copy()
X_ols = XM_base.join(X_country, rsuffix='cn').join(X_lang, rsuffix='lg').join(X_vecs)

ols = sm.OLS(y_ols, X_ols) 
reg_res = ols.fit().get_robustcov_results()

loc_DFem = X_ols.columns.get_loc('D_Female')
reg_meta_ols_results.update({'spec3': store_ols_results(reg_res,loc_DFem)})

In [79]:
# Companies with at least 10 movies
X_company = pd.get_dummies(df_scores['company'])
X_company = X_company[X_company.columns[X_company.sum()>=10]]

# Distributors with at least 10 movies
X_dist = pd.get_dummies(df_scores['Distributor'])
X_dist = X_dist[X_dist.columns[X_dist.sum()>=10]]



In [80]:
# spec4:  spec3 + company and distributor FE
y_ols = y_meta.copy()
X_ols = XM_base.join(X_country, rsuffix='cn').join(X_lang, rsuffix='lg').\
        join(X_vecs).join(X_company, rsuffix='cp').\
        join(X_dist, rsuffix='ds')

ols = sm.OLS(y_ols, X_ols) 
reg_res = ols.fit().get_robustcov_results()

loc_DFem = X_ols.columns.get_loc('D_Female')
reg_meta_ols_results.update({'spec4': store_ols_results(reg_res,loc_DFem)})

In [81]:
#Cast
X_cast = pd.DataFrame(mlb.fit_transform(df_scores['cast']),columns=mlb.classes_, index=df_scores.index)
# Keeping actors whose names showed up at least 10 times
X_cast = X_cast[X_cast.columns[X_cast.sum()>=10]]

# Age Ratings Categories
X_mpaa = df_scores[['rated_R','rated_PG', 'rated_PG13', 'rated_TVMA', 'rated_TV14']]

# Genres
X_genre = df_scores[genres]

In [82]:
# spec5:  spec4 + cast + age ratigns, mpaa, and genres
y_ols = y_meta.copy()
X_ols = XM_base.join(X_country, rsuffix='cn').join(X_lang, rsuffix='lg').\
        join(X_vecs).join(X_cast, rsuffix='cs').join(X_company, rsuffix='cp').\
        join(X_dist, rsuffix='ds').join(X_mpaa).join(X_genre, rsuffix='gn')

ols = sm.OLS(y_ols, X_ols) 
reg_res = ols.fit().get_robustcov_results()

loc_DFem = X_ols.columns.get_loc('D_Female')
reg_meta_ols_results.update({'spec5': store_ols_results(reg_res,loc_DFem)})

### DML

In [83]:

XM_full = XM_base.join(X_country, rsuffix='cn').join(X_lang, rsuffix='lg').\
        join(X_vecs).join(X_cast, rsuffix='cs').join(X_company, rsuffix='cp').\
        join(X_dist, rsuffix='ds').join(X_mpaa).join(X_genre, rsuffix='gn')

data_meta = XM_full.join(y_meta)
y_meta = data_meta['metascore']
x_cols = list(set(data_meta.columns.values) - {'D_Female','metascore' })

dml_data_meta = DoubleMLData(data_meta, y_col = 'metascore', d_cols = 'D_Female',
                        x_cols = x_cols)

print(f'Number of Features: {len(x_cols)}')

Number of Features: 1224


In [84]:
# Tuning Classifier (learner_m):
from sklearn.model_selection import GridSearchCV
# Selection Stage
d_fem = XM_full['D_Female']
X_cls = XM_full.drop(columns=['D_Female'])

# # Logit
# param_grid = {'penalty': ['l2', 'l1'], 
#               'C': [ 1e5,1e6, 1e7, 1e8, 1e9]}
# lgt = LogisticRegression(solver='liblinear', random_state=seed)
# clf = GridSearchCV(lgt, param_grid, scoring='f1',cv = ShuffleSplit(n_splits=5,random_state=seed))
# clf.fit(X_cls, d_fem)
# lgt_classifier = clf.best_estimator_
# lgt_params = {'C': [clf.best_params_['C']], 'penalty': [clf.best_params_['penalty']]}

# # F1 Test Scores are about 0.20
# lgt_test_scores = cross_val_score(lgt_classifier,X_cls, d_fem, n_jobs=-1, scoring = 'f1',cv = ShuffleSplit(n_splits=5))
# print(f'lgt_scores: {lgt_test_scores}')

In [85]:
# Since tuning models in DML takes time, I defined the tuned versions from the previous runs
lgt_classifier = LogisticRegression(solver='liblinear', penalty='l2', C=1e8,  random_state=seed)

ENet_tuned = ElasticNet(alpha=0.001, l1_ratio=0.01)

RF_tuned = RandomForestRegressor(n_jobs = -1, n_estimators = 300, max_features = 'sqrt'
                          , max_depth=40, min_samples_split= 8, random_state=seed)

XGB_tuned = XGBRegressor(objective = "reg:squarederror", eta = 0.2,
                        n_estimators =50, n_jobs=-1, max_depth= 10, random_state=seed)

In [86]:
# # Random Forests
# rfc_scores = cross_val_score(RF_tuned,XM_full, y_meta, n_jobs=-1, scoring = 'r2',
#                              cv = ShuffleSplit(n_splits=3))
# print(f'rfc_scores: {rfc_scores}')

# # XGBoost: TFIDF Matrix
# xgb_scores =  cross_val_score(XGB_tuned,XM_full, y_meta, n_jobs=-1, scoring = 'r2',
#                               cv = ShuffleSplit(n_splits=3))
# print(f'xgb_scores: {xgb_scores}')

# # Elastic Net: 
# elnet_scores = cross_val_score(ENet_tuned,XM_full, y_meta, n_jobs=-1, scoring = 'r2',cv = ShuffleSplit(n_splits=5))
# print(f'elnet_scores: {elnet_scores}')

# # CV R2 Summary: RF roughly 0.45, XGB roughly 0.50, Elastic Net: roughly 0.63

In [87]:
# DML 1
np.random.seed(seed)
# learner_g = ElasticNet()
learner_g = ENet_tuned
learner_m = lgt_classifier
ml_g = clone(learner_g)
ml_m = clone(learner_m)
param_grids = {'ml_g': {'alpha': [0.0001, 0.001, 0.1], 'l1_ratio': [ 0.01,0.25,0.5,0.75,0.99]},
              'ml_m': lgt_params}
dml_meta = DoubleMLPLR(dml_data_meta, ml_g, ml_m)
# dml_meta.tune(param_grids, n_folds_tune=3, n_jobs_cv=-1)
dml_meta.fit(n_jobs_cv=-1, store_predictions=True);

# Store Coef and SE
reg_meta_dml_results.update({'ElasticNet': store_dml_results(dml_meta)})

In [88]:
# DML Random Forest
np.random.seed(seed)
# learner_g = RandomForestRegressor(n_jobs = -1, n_estimators = 300,
#                                   max_features= 'sqrt')
learner_g = RF_tuned
learner_m = lgt_classifier
ml_g = clone(learner_g)
ml_m = clone(learner_m)

param_grids = {'ml_g': {'max_depth': [ 20, 40, 80], 'min_samples_split': [4,8,16]},
              'ml_m': lgt_params}

dml_meta = DoubleMLPLR(dml_data_meta, ml_g, ml_m)
# dml_meta.tune(param_grids, n_folds_tune=3, n_jobs_cv=-1)
dml_meta.fit(n_jobs_cv=-1, store_predictions=True);

# Store Coef and SE
reg_meta_dml_results.update({'RandomForest': store_dml_results(dml_meta)})

In [89]:
# DML XGBoost
np.random.seed(seed)
# learner_g =  XGBRegressor(objective = "reg:squarederror",
#                         n_estimators =50, n_jobs=-1, random_state=seed)
learner_g = XGB_tuned
learner_m = lgt_classifier
ml_g = clone(learner_g)
ml_m = clone(learner_m)
param_grids = {'ml_g': {'max_depth': [10, 20, 30], 'eta': [0.2, 0.4, 0.6]},
              'ml_m': lgt_params}

dml_meta = DoubleMLPLR(dml_data_meta, ml_g, ml_m)
# dml_meta.tune(param_grids, n_folds_tune=3, n_jobs_cv=-1)
dml_meta.fit(n_jobs_cv=-1, store_predictions=True);

# Store Coef and SE
reg_meta_dml_results.update({'XGBoost': store_dml_results(dml_meta)})

/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:703: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


### MeToo Impact

In [90]:
df_scores_me2= df_scores.copy()
#post me too >=2018
df_scores_me2['DFem_PMe2'] = np.where(((df_scores_me2['D_Female']==1) & (df_scores_me2['Year']>2017)), 1,0)

In [91]:
# OLS
# XS_base: X_base of the Sales specification
y_meta, XM_me2 = dmatrices('metascore ~ D_Female + C(Year) + DFem_PMe2',
                    data=df_scores_me2, return_type='dataframe')

# OLS: Fully Saturated Model
y_ols = y_meta.copy()
X_ols = XM_me2.join(X_country, rsuffix='cn').join(X_lang, rsuffix='lg').\
        join(X_vecs).join(X_company, rsuffix='cp').\
        join(X_dist, rsuffix='ds')

ols = sm.OLS(y_ols, X_ols) 
reg_res = ols.fit().get_robustcov_results()

loc_DFem = X_ols.columns.get_loc('D_Female')
loc_DFem_pme2 = X_ols.columns.get_loc('DFem_PMe2')
reg_meta_ols_results.update({'metoo': 
                      store_ols_results(reg_res,loc_DFem,loc_DFem_pme2,
                                        post_me2=True)})

In [92]:
# DML
data_meta_me2 = X_ols.join(y_meta)
x_cols_me2 = list(set(X_ols.columns.values) - {'D_Female', 'DFem_PMe2'})

dml_data_meta_me2 = DoubleMLData(data_meta_me2, y_col = 'metascore', d_cols =
                                 ['D_Female','DFem_PMe2'], x_cols = x_cols_me2)

# Double ML:
# DML ElasticNet
learner_g = ENet_tuned
learner_m = lgt_classifier
ml_g = clone(learner_g)
ml_m = clone(learner_m)
dml_meta = DoubleMLPLR(dml_data_meta_me2, ml_g, ml_m)
dml_meta.fit(n_jobs_cv=-1, store_predictions=True);
reg_meta_dml_results.update({'metoo(ENet)': store_dml_results(dml_meta, 
                                                                   post_me2=True)})

### Save Results

In [124]:
#Save Results
reg_meta_results = {'OLS': reg_meta_ols_results, 'DML': reg_meta_dml_results}
agg_outcomes_reg_results.update({'Meta': reg_meta_results})

with open(results_dir + 'agg_outcomes_reg_results.json', 'w') as outfile:
    json.dump(agg_outcomes_reg_results, outfile)

In [94]:
# Create DataFrame of the Results

specs = list(reg_meta_ols_results.keys()) + list(reg_meta_dml_results.keys()) 

params_ols =  ['coef D_Female', 'se D_Female', 'p-val D_Female']
params_dml =  ['coef D_Female', 'se D_Female', 'p-val D_Female']
spec_ols = list(reg_meta_ols_results.keys() -{'metoo'})
spec_dml = list(reg_meta_dml_results.keys() - {'metoo(ENet)'} )

data_ols = [[reg_meta_ols_results[spec][p][0] for p in params_ols] for spec in spec_ols]
data_r2 = [[reg_meta_ols_results[spec][p] for p in ['R2', 'N Obs']] for spec in spec_ols]
data_ols = [data_ols[r] + data_r2[r] for r in range(len(data_ols))]

data_dml = [[reg_meta_dml_results[spec][p][0] for p in params_dml] for spec in spec_dml]




data_DFem = np.asarray(data_ols + data_dml )


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [95]:
# Cast  the results in tables
table_ols_meta = pd.DataFrame(np.asarray(data_ols).T, columns= spec_ols,
                               index = ['coef', 'SE', 'P-Val', 'R2', 'N'])

table_dml_meta = pd.DataFrame(np.asarray(data_dml).T, columns= spec_dml,
                               index = ['coef', 'SE', 'P-Val'])

## Userscore

In [96]:
# Temporarily Storing Results in dicts
reg_user_dml_results={}
reg_user_ols_results = {}

### OLS

In [97]:
# XS_base: X_base of the user specification
y_user, XU_base = dmatrices('userscore ~  D_Female + C(Year)',
                    data=df_scores, return_type='dataframe')

# XGB throws error if '[]' are in columns names:
year_cols = dict(zip([f'C(Year)[T.{y}]' for y in range(1991,2022)],
                     [f'Year({y})' for y in range(1991,2022)] ))
XU_base = XU_base.rename(columns = year_cols)

In [98]:
# OLS user Baseline
y_ols = y_user.copy()
X_ols = XU_base.copy()
ols = sm.OLS(y_ols, X_ols) 
reg_res = ols.fit().get_robustcov_results()

loc_DFem = X_ols.columns.get_loc('D_Female')
reg_user_ols_results.update({'Base': store_ols_results(reg_res,loc_DFem)})

In [99]:
#Countries
df_scores['country_list'] = df_scores['country'].map(lambda x: re.sub(' ','',x)).\
                        map(lambda x: x.split(','))

X_country = pd.DataFrame(mlb.fit_transform(df_scores['country_list']),
             columns=mlb.classes_, index=df_scores.index)

X_country = X_country[X_country.columns[X_country.sum()>=10]]

#Language
df_scores['lang_list'] = df_scores['language'].map(lambda x: re.sub(' ','',x)).\
                        map(lambda x: x.split(','))
X_lang = pd.DataFrame(mlb.fit_transform(df_scores['lang_list']),
             columns=mlb.classes_, index=df_scores.index)
X_lang = X_lang[X_lang.columns[X_lang.sum()>=10]]

In [100]:
# OLS Country and Language FE: Spec2
y_ols = y_user.copy()
X_ols = XU_base.join(X_country, rsuffix='cn').join(X_lang, rsuffix='lg')

ols = sm.OLS(y_ols, X_ols) 
reg_res = ols.fit().get_robustcov_results()

loc_DFem = X_ols.columns.get_loc('D_Female')
reg_user_ols_results.update({'spec2': store_ols_results(reg_res,loc_DFem)})

In [101]:
# Plot Truncated SVD Vectors
plot_vectors = [f'plot_vec{i}' for i in range(100)]
X_vecs = df_scores[plot_vectors]

In [102]:
# spec3 = spec2 + Plot Vectors FE
y_ols = y_user.copy()
X_ols = XU_base.join(X_country, rsuffix='cn').join(X_lang, rsuffix='lg').join(X_vecs)

ols = sm.OLS(y_ols, X_ols) 
reg_res = ols.fit().get_robustcov_results()

loc_DFem = X_ols.columns.get_loc('D_Female')
reg_user_ols_results.update({'spec3': store_ols_results(reg_res,loc_DFem)})

In [103]:
# Companies with at least 10 movies
X_company = pd.get_dummies(df_scores['company'])
X_company = X_company[X_company.columns[X_company.sum()>=10]]

# Distributors with at least 10 movies
X_dist = pd.get_dummies(df_scores['Distributor'])
X_dist = X_dist[X_dist.columns[X_dist.sum()>=10]]



In [104]:
# spec4:  spec3 + company and distributor FE
y_ols = y_user.copy()
X_ols = XU_base.join(X_country, rsuffix='cn').join(X_lang, rsuffix='lg').\
        join(X_vecs).join(X_company, rsuffix='cp').\
        join(X_dist, rsuffix='ds')

ols = sm.OLS(y_ols, X_ols) 
reg_res = ols.fit().get_robustcov_results()

loc_DFem = X_ols.columns.get_loc('D_Female')
reg_user_ols_results.update({'spec4': store_ols_results(reg_res,loc_DFem)})

In [105]:
#Cast
X_cast = pd.DataFrame(mlb.fit_transform(df_scores['cast']),columns=mlb.classes_, index=df_scores.index)
# Keeping actors whose names showed up at least 10 times
X_cast = X_cast[X_cast.columns[X_cast.sum()>=10]]


# Age Ratings Categories
X_mpaa = df_scores[['rated_R','rated_PG', 'rated_PG13', 'rated_TVMA', 'rated_TV14']]

# Genres
X_genre = df_scores[genres]

In [106]:
# spec5:  spec4 + cast + age ratigns, mpaa, and genres
y_ols = y_user.copy()
X_ols = XU_base.join(X_country, rsuffix='cn').join(X_lang, rsuffix='lg').\
        join(X_vecs).join(X_cast, rsuffix='cs').join(X_company, rsuffix='cp').\
        join(X_dist, rsuffix='ds').join(X_mpaa).join(X_genre, rsuffix='gn')

ols = sm.OLS(y_ols, X_ols) 
reg_res = ols.fit().get_robustcov_results()

loc_DFem = X_ols.columns.get_loc('D_Female')
reg_user_ols_results.update({'spec5': store_ols_results(reg_res,loc_DFem)})

### DML

In [107]:

XU_full = XU_base.join(X_country, rsuffix='cn').join(X_lang, rsuffix='lg').\
        join(X_vecs).join(X_cast, rsuffix='cs').join(X_company, rsuffix='cp').\
        join(X_dist, rsuffix='ds').join(X_mpaa).join(X_genre, rsuffix='gn')

data_user = XU_full.join(y_user)
y_user = data_user['userscore']
x_cols = list(set(data_user.columns.values) - {'D_Female','userscore' })

dml_data_user = DoubleMLData(data_user, y_col = 'userscore', d_cols = 'D_Female',
                        x_cols = x_cols)

print(f'Number of Features: {len(x_cols)}')

Number of Features: 1224


In [108]:
# Logit Classfier from the Previous part still applies

In [109]:
# Since tuning models in DML takes time, I defined the tuned versions from the previous runs
lgt_classifier = LogisticRegression(solver='liblinear', penalty='l2', C=1e8,  random_state=seed)

ENet_tuned = ElasticNet(alpha=0.001, l1_ratio=0.01)

RF_tuned = RandomForestRegressor(n_jobs = -1, n_estimators = 300, max_features = 'sqrt'
                          , max_depth=40, min_samples_split= 4, random_state=seed)

XGB_tuned = XGBRegressor(objective = "reg:squarederror", eta = 0.2,
                        n_estimators =50, n_jobs=-1, max_depth= 10, random_state=seed)

In [110]:
# # Random Forests
# rfc_scores = cross_val_score(RF_tuned,XU_full, y_user, n_jobs=-1, scoring = 'r2',
#                              cv = ShuffleSplit(n_splits=3))
# print(f'rfc_scores: {rfc_scores}')

# # XGBoost: TFIDF Matrix
# xgb_scores =  cross_val_score(XGB_tuned,XU_full, y_user, n_jobs=-1, scoring = 'r2',
#                               cv = ShuffleSplit(n_splits=3))
# print(f'xgb_scores: {xgb_scores}')

# # Elastic Net: 
# elnet_scores = cross_val_score(ENet_tuned,XU_full, y_user, n_jobs=-1, scoring = 'r2',cv = ShuffleSplit(n_splits=5))
# print(f'elnet_scores: {elnet_scores}')

# # CV R2 Summary: RF roughly 0.45, XGB roughly 0.50, Elastic Net: roughly 0.63

In [111]:
# DML 1
np.random.seed(seed)
# learner_g = ElasticNet()
learner_g = ENet_tuned
learner_m = lgt_classifier
ml_g = clone(learner_g)
ml_m = clone(learner_m)
param_grids = {'ml_g': {'alpha': [0.0001, 0.001, 0.1], 'l1_ratio': [ 0.01,0.25,0.5,0.75,0.99]},
              'ml_m': lgt_params}
dml_user = DoubleMLPLR(dml_data_user, ml_g, ml_m)
# dml_user.tune(param_grids, n_folds_tune=3, n_jobs_cv=-1)
dml_user.fit(n_jobs_cv=-1, store_predictions=True);

# Store Coef and SE
reg_user_dml_results.update({'ElasticNet': store_dml_results(dml_user)})

In [112]:
# DML Random Forest
np.random.seed(seed)
# learner_g = RandomForestRegressor(n_jobs = -1, n_estimators = 300,
#                                   max_features= 'sqrt')
learner_g = RF_tuned
learner_m = lgt_classifier
ml_g = clone(learner_g)
ml_m = clone(learner_m)

param_grids = {'ml_g': {'max_depth': [ 20, 40, 80], 'min_samples_split': [4,8,16]},
              'ml_m': lgt_params}

dml_user = DoubleMLPLR(dml_data_user, ml_g, ml_m)
# dml_user.tune(param_grids, n_folds_tune=3, n_jobs_cv=-1)
dml_user.fit(n_jobs_cv=-1, store_predictions=True);

# Store Coef and SE
reg_user_dml_results.update({'RandomForest': store_dml_results(dml_user)})

In [113]:
# DML XGBoost
np.random.seed(seed)
# learner_g =  XGBRegressor(objective = "reg:squarederror",
#                         n_estimators =50, n_jobs=-1, random_state=seed)
learner_g = XGB_tuned
learner_m = lgt_classifier
ml_g = clone(learner_g)
ml_m = clone(learner_m)
param_grids = {'ml_g': {'max_depth': [10, 20, 30], 'eta': [0.2, 0.4, 0.6]},
              'ml_m': lgt_params}

dml_user = DoubleMLPLR(dml_data_user, ml_g, ml_m)
# dml_user.tune(param_grids, n_folds_tune=3, n_jobs_cv=-1)
dml_user.fit(n_jobs_cv=-1, store_predictions=True);

# Store Coef and SE
reg_user_dml_results.update({'XGBoost': store_dml_results(dml_user)})

### MeToo Impact

In [114]:
# OLS
# XS_base: X_base of the Sales specification
y_user, XU_me2 = dmatrices('userscore ~ D_Female + C(Year) + DFem_PMe2',
                    data=df_scores_me2, return_type='dataframe')

# OLS: Fully Saturated Model
y_ols = y_user.copy()
X_ols = XM_me2.join(X_country, rsuffix='cn').join(X_lang, rsuffix='lg').\
        join(X_vecs).join(X_company, rsuffix='cp').\
        join(X_dist, rsuffix='ds')

ols = sm.OLS(y_ols, X_ols) 
reg_res = ols.fit().get_robustcov_results()

loc_DFem = X_ols.columns.get_loc('D_Female')
loc_DFem_pme2 = X_ols.columns.get_loc('DFem_PMe2')
reg_user_ols_results.update({'metoo': 
                      store_ols_results(reg_res,loc_DFem,loc_DFem_pme2,
                                        post_me2=True)})

In [115]:
# DML
data_user_me2 = X_ols.join(y_user)
x_cols_me2 = list(set(X_ols.columns.values) - {'D_Female', 'DFem_PMe2'})

dml_data_user_me2 = DoubleMLData(data_user_me2, y_col = 'userscore', d_cols =
                                 ['D_Female','DFem_PMe2'], x_cols = x_cols_me2)

# Double ML:
# DML ElasticNet
learner_g = ENet_tuned
learner_m = lgt_classifier
ml_g = clone(learner_g)
ml_m = clone(learner_m)
dml_user = DoubleMLPLR(dml_data_user_me2, ml_g, ml_m)
dml_user.fit(n_jobs_cv=-1, store_predictions=True);
reg_user_dml_results.update({'metoo(ENet)': store_dml_results(dml_user, 
                                                                   post_me2=True)})

### Save Results

In [125]:
#Save Results
reg_user_results = {'OLS': reg_user_ols_results, 'DML': reg_user_dml_results}
agg_outcomes_reg_results.update({'User': reg_user_results})

with open(results_dir + 'agg_outcomes_reg_results.json', 'w') as outfile:
    json.dump(agg_outcomes_reg_results, outfile)

In [117]:
# Create DataFrame of the Results

specs = list(reg_user_ols_results.keys()) + list(reg_user_dml_results.keys()) 

params_ols =  ['coef D_Female', 'se D_Female', 'p-val D_Female']
params_dml =  ['coef D_Female', 'se D_Female', 'p-val D_Female']
spec_ols = list(reg_user_ols_results.keys() -{'metoo'})
spec_dml = list(reg_user_dml_results.keys() - {'metoo(ENet)'} )

data_ols = [[reg_user_ols_results[spec][p][0] for p in params_ols] for spec in spec_ols]
data_r2 = [[reg_user_ols_results[spec][p] for p in ['R2', 'N Obs']] for spec in spec_ols]
data_ols = [data_ols[r] + data_r2[r] for r in range(len(data_ols))]

data_dml = [[reg_user_dml_results[spec][p][0] for p in params_dml] for spec in spec_dml]


data_DFem = np.asarray(data_ols + data_dml )

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [118]:
# Cast the reults in tables
table_ols_user = pd.DataFrame(np.asarray(data_ols).T, columns= spec_ols,
                               index = ['coef', 'SE', 'P-Val', 'R2', 'N'])

table_dml_user = pd.DataFrame(np.asarray(data_dml).T, columns= spec_dml,
                               index = ['coef', 'SE', 'P-Val'])

## Regression Results Tables

### Without Post×MeToo

In [119]:
table_results_ols = pd.concat([table_ols_sales, table_ols_budget,
                               table_ols_meta, table_ols_user],
          axis=0, keys=['sales', 'budget', 'metascore', 'userscore'])

table_results_dml = pd.concat([table_dml_sales, table_dml_budget,
                               table_dml_meta, table_dml_user],
          axis=0, keys=['sales', 'budget' , 'metascore', 'userscore'])

table_results_ols.to_csv(results_dir + 'reg_results_ols.csv')
table_results_dml.to_csv(results_dir + 'reg_results_dml.csv')

### With Post×MeToo

In [120]:
data_ols = []
data_ols_me2 = []
data_r2 = []
spec_ols_me2 = ['metoo']
ols_results = [reg_sales_ols_results, reg_budget_ols_results, reg_meta_ols_results, 
               reg_user_ols_results]

for result in ols_results:
  data_ols = data_ols + [[result[spec][p][0] for p in params_ols] + 
                [result[spec][p][1] for p in params_ols] 
                for spec in spec_ols_me2]

  data_r2 = data_r2 + [[result[spec][p] for p in ['R2', 'N Obs']] 
                       for spec in spec_ols_me2]

  data_ols_me2 = [data_ols[r] + data_r2[r] for r in range(len(data_ols))]



# data_dml_pm2 = [[reg_user_dml_results[spec][p][0] for p in params_dml] +
#                 [reg_user_dml_results[spec][p][1] for p in params_dml]
#                 for spec in spec_dml_me2]

In [121]:
data_dml_me2 = []
spec_dml_me2 = ['metoo(ENet)']
dml_results = [reg_sales_dml_results, reg_budget_dml_results, reg_meta_dml_results, 
               reg_user_dml_results]

for result in dml_results:
  data_dml_me2 = data_dml_me2 + [[result[spec][p][0] for p in params_dml] + 
                [result[spec][p][1] for p in params_dml] 
                for spec in spec_dml_me2]


In [122]:
columns = ['Sales', 'Budget', 'Meta', 'User']
index_ols = ['coef', 'SE', 'P-Val', 'coef PMe2', 'SE PMe2', 'P-Val PMe2', 'R2', 'N']
index_dml = ['coef', 'SE', 'P-Val', 'coef PMe2', 'SE PMe2', 'P-Val PMe2']


table_ols_me2 = pd.DataFrame(np.asarray(data_ols_me2).T, columns= columns,
                               index = index_ols)

table_dml_me2 = pd.DataFrame(np.asarray(data_dml_me2).T, columns= columns,
                               index = index_dml)

table_ols_me2.to_csv(results_dir + 'reg_results_me2_ols.csv')
table_dml_me2.to_csv(results_dir + 'reg_results_me2_dml.csv')